In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import cv2

In [ ]:
from skimage import data, color, feature
import skimage.data

image = color.rgb2gray(data.chelsea())
hog_vec, hog_vis = feature.hog(image, visualize=True)
print(len(hog_vec))
fig, ax = plt.subplots(1, 2, figsize=(12, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
ax[0].imshow(image, cmap='gray')
ax[0].set_title('input image')

ax[1].imshow(hog_vis)
ax[1].set_title('visualization of HOG features');

In [ ]:
from google.colab import drive
import cv2
drive.mount('/content/drive')
dataset_folder = "/content/drive/MyDrive/haascades/"
face_cascade = cv2.CascadeClassifier(dataset_folder + 'haarcascade_frontalface_default.xml')

In [ ]:
from sklearn.datasets import fetch_lfw_people
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.datasets import fetch_olivetti_faces
import numpy as np
from skimage.transform import resize

# Load the LFW dataset
faces1 = fetch_lfw_people()

# Load the Olivetti Faces dataset
faces2 = fetch_olivetti_faces()

# Extract images from both datasets
patches1 = faces1.images
patches2 = faces2.images

desired_shape = (45, 45)
positive_patches1_resized = [resize(image, desired_shape) for image in patches1]
positive_patches2_resized = [resize(image, desired_shape) for image in patches2]

# Concatenate the resized images from both datasets along the first axis (axis=0)
positive_patches = np.concatenate([positive_patches1_resized, positive_patches2_resized], axis=0)



In [ ]:
# method 1

from PIL import Image
import os
from skimage import data, color, feature
import skimage.data
import numpy as np
import cv2
from google.colab import drive

drive.mount('/content/drive')

dataset_folder = "/content/drive/MyDrive/haascades/"
input_directory = '/content/drive/MyDrive/extracted_images/'
output_directory = '/content/drive/MyDrive/processed_images1/'

face_cascade = cv2.CascadeClassifier(dataset_folder + 'haarcascade_frontalface_default.xml')

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def detect(img):
  faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(45, 45))
  try:
    x, y, w, h = faces[0]
    img = img[y:y+h, x:x+w]
    img = Image.fromarray(img)
    img = img.resize((45, 45))
  except:
    img = None
  return img

_imgfile = []

def batch_process_images(input_directory, output_directory):
    for filename in os.listdir(input_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_path = os.path.join(input_directory, filename)
            output_path = os.path.join(output_directory, filename)
            try:
                with Image.open(input_path) as img:
                    img = color.rgb2gray(img)
                    img = (img * 255).astype(np.uint8)
                    img = detect(img)
                    _imgfile.append(img)
                    img.save(output_path)
                    try:
                      print("Thanh cong_Successfully: "% filename)
                    except AttributeError:
                      print('\n')
            except Exception as e:
                print(f"Error processing {filename}: {e}")

batch_process_images(input_directory, output_directory)

In [ ]:
# cách 2

from PIL import Image
import os
from skimage import data, color, feature
import skimage.data
import numpy as np
import cv2
import zipfile

drive.mount('/content/drive')

path_imgidx = "/content/drive/MyDrive/zip/CASIA-WebFace_crop.zip"
imgzip = zipfile.ZipFile(path_imgidx)
inflist = imgzip.infolist()

image_file = []
cropped_img = []
crop = []
X_train = []
npoint = 8
rad = 1.0

def detect(img, idx):
  faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(45, 45))
  try:
    x, y, w, h = faces[0]
    img = img[y:y+h, x:x+w]
    img = cv2.resize(img, (45,45))
  except:
    img = None
  return img


for f in inflist:
    ifile = imgzip.open(f)
    img = Image.open(ifile)
    img = color.rgb2gray(img)
    img = (img * 255).astype(np.uint8)
    image_file.append(img)

for i, img in enumerate(image_file):
    img = detect(img, i)
    if img is not None:
      img = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
      cropped_img.append(img)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n_arr = []
for i in range(len(processed_image)):
  arr = np.array(processed_image[i])
  imgs = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
  n_arr.append(imgs)

plt.imshow(n_arr[0])

In [ ]:
positive_patches = np.concatenate([positive_patches, cropped_img],axis=0)

In [ ]:
from skimage import data, transform

imgs_to_use = ['camera', 'text', 'coins', 'moon',
               'page', 'clock', 'immunohistochemistry',
               'chelsea', 'coffee', 'hubble_deep_field']
images = [getattr(data, name)()
          for name in imgs_to_use]
for i, data in enumerate(images):
  if len(data.shape) > 2:
    images[i] = color.rgb2gray(data)


In [ ]:
from sklearn.feature_extraction.image import PatchExtractor

def extract_patches(img, N, scale=1.0, patch_size=positive_patches[0].shape):
    extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches

negative_patches = np.vstack([extract_patches(im, 1000, scale)
                              for im in images for scale in [0.5, 1.0, 2.0]])

In [ ]:
import zipfile
from PIL import Image
from google.colab import drive
import os
drive.mount('/content/drive')
path_imgidx = "/content/drive/MyDrive/zip/CASIA-WebFace_crop.zip"
imgzip = zipfile.ZipFile(path_imgidx)
inflist = imgzip.infolist()

image_file = []
cropped_img = []

def detect(img, idx):
  faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(45, 45))
  try:
    x, y, w, h = faces[0]
    img = img[y:y+h, x:x+w]
    img = cv2.resize(img, (45,45))
  except:
    img = None
  return img

for f in inflist:
    ifile = imgzip.open(f)
    img = Image.open(ifile)
    img = color.rgb2gray(img)
    img = (img * 255).astype(np.uint8)
    image_file.append(img)

for i, img in enumerate(image_file):
    img = detect(img, i)
    if img is not None:
      cropped_img.append(img)

In [ ]:
from itertools import chain
from sklearn.model_selection import train_test_split

_train = []

npoint = 8
rad = 1.0
for im in chain(positive_patches, negative_patches):
    # Compute LBP features
    lbp_feature = feature.local_binary_pattern(im, npoint, rad)
    lbp_feature = np.histogram(lbp_feature.ravel(), bins=np.arange(0, npoint + 3), range=(0, npoint + 2))
    lbp_feature = lbp_feature[0]

    # Compute HOG features
    hog_feature = feature.hog(im)

    # Concatenate LBP and HOG features
    concatenated_feature = np.concatenate((hog_feature, lbp_feature))
    _train.append(concatenated_feature)

X = np.array(_train)
y = np.zeros(X.shape[0])
y[:positive_patches.shape[0]] = 1

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=5000)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(LinearSVC(), {'C': [1.0, 2.0, 4.0, 8.0]})
grid.fit(X_train, y_train)
grid.best_score_

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

model = make_pipeline(StandardScaler(), grid.best_estimator_)
# model = make_pipeline(StandardScaler(), SVC(C=1, gamma='auto', probability=True))
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

In [ ]:
joblib.dump(model, "best_model.pkl")

In [ ]:
img = cv2.imread("sample_data/1.jpg")
test_image = img
test_image = skimage.color.rgb2gray(test_image)
test_image = skimage.transform.rescale(test_image, 0.5)

plt.imshow(test_image, cmap='gray')
plt.axis('off');

In [ ]:
def sliding_window(img, patch_size=positive_patches[0].shape,
                   istep=2, jstep=2, scale=1.0):
    Ni, Nj = (int(scale * s) for s in patch_size)
    for i in range(0, img.shape[0] - Ni, istep):
        for j in range(0, img.shape[1] - Ni, jstep):
            patch = img[i:i + Ni, j:j + Nj]
            if scale != 1:
                patch = transform.resize(patch, patch_size)
            yield (i, j), patch

In [ ]:
indices, patches = zip(*sliding_window(test_image))
# patches_hog = np.array([feature.hog(patch) for patch in patches])


npoint = 8
rad = 1.0
patches_hog = []
for im in patches:
    print(im.shape)
    # Compute LBP features
    lbp_feature = feature.local_binary_pattern(im, npoint, rad)
    lbp_feature = np.histogram(lbp_feature.ravel(), bins=np.arange(0, npoint + 3), range=(0, npoint + 2))
    lbp_feature = lbp_feature[0]

    # Compute HOG features
    hog_feature = feature.hog(im)

    # Concatenate LBP and HOG features

    concatenated_feature = np.concatenate((hog_feature, lbp_feature))
    result = model.predict_proba([concatenated_feature])

    # Append to the list of feature vectors
    patches_hog.append(concatenated_feature)

patches_hog = np.array(patches_hog)


# patches_hog.shape
confidence_scores = model.decision_function(patches_hog)
labels = model.predict(patches_hog)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray')
ax.axis('off')

Ni, Nj = positive_patches[0].shape

indices = np.array(indices)

for i, j in indices[labels == 1]:
    ax.add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red',
                               alpha=0.3, lw=2, facecolor='none'))

